# Chopin Nocturnes
## Comparing Generated Sequences to Training Data

In [1]:
from music_generator.serializers.discrete_time_serializer import DiscreteTimeMidiSerializer
import music_generator.utilities.sequence_utils as sequence_utils

from pathlib import Path

### Set up constants for generation and comparison

A single quarter-note four-voice chord would be made up of at least 9 events (4 note-on, 4 note-off, at least 4 wait events if no chaining is required),
so a measure of four quarter-note four-voice chords would be a minimum of 36 events.

Adding in eigth notes, flourishes, etc. would increase the number of events in a measure.

Windows of 100 events are used to compare generated sequences and training sequences. This would generally represent somewhere from one to four measures, depending on note frequency.

In [2]:
serializer = DiscreteTimeMidiSerializer()
window_size = 100

### Create a set of all unique sub-sequences with length = window_size from the training data

The sequences from the training data are transposed over one octave.

These sequences are then windowed to the previously decided number of events.

Each window is converted to an equivalent string to make it a hashable object, and then added to a set for comparison with another set.

In [3]:
real_sequences = serializer.serialize_folder('./training_data/chopin_nocturnes/')
# transpose training data to all keys
real_sequences = sequence_utils.transpose(real_sequences, down=-6, up=5)
real_sequences, _ = sequence_utils.window(real_sequences, window_size=window_size)
print('Training data windows of length {}: {}'.format(window_size, len(real_sequences)))

real_set = set()
for s in real_sequences:
    s = '-'.join([str(x) for x in s])
    real_set.add(s)
print('Unique windows: {}'.format(len(real_set)))

Training data windows of length 100: 1555488
Unique windows: 1482911


### Compare generated sequences to the training data
Sequences generated by the model using different seeds and temperature settings are serialized, converted to a hashable string, added to a set, and then compared to the training data one at a time.
The percentage of unique windows in a generated composition that appear exactly in the training data is calculated.
The average percentage of 'copied' sequences for an entire set of compositions using a given temperature setting is also calculated.

In [4]:
temperatures = [0.5, 1.0, 1.2, 1.5, 2.0, 3.0]

for temp in temperatures:
    print('TEMPERATURE: {}'.format(temp))
    
    percentages = []

    # compare each generated sequence against the training set
    for file in Path('./generated_files/chopin_nocturnes_temperature_{}/'.format(temp)).glob('*.mid'):
        
        sequence = serializer.serialize(file)
            
        # split generated sequence into subsequences
        gen_sequences, _ = sequence_utils.window([sequence], window_size=window_size)

        # create a set of unique subsequences
        gen_set = set()
        for s in gen_sequences:
            # turn subsequence into string so it is hashable
            s = '-'.join([str(x) for x in s])
            gen_set.add(s)

        # find the intersection of the two sets to find all matching subsequences and calculate percentage of generated subsequences that come from the training data
        matches = gen_set.intersection(real_set)
        n_matches = len(matches)
        total = len(gen_set)
        percentage = n_matches/total * 100
        percentages.append(percentage)

        # print results
        print('{}: {:.2f}% of unique windows (length = {}) from the generated composition exist in the training data.'.format(file.name, percentage, window_size))
                        
    average = sum(percentages) / len(percentages)
    print('AVERAGE PERCENTAGE OVER ALL FILES: {:.2f}%'.format(average))
    print('*' * 80)


TEMPERATURE: 0.5
sample_45-60-64-69.mid: 61.12% of unique windows (length = 100) from the generated composition exist in the training data.
sample_38-47-54-62-66.mid: 2.27% of unique windows (length = 100) from the generated composition exist in the training data.
sample_95.mid: 39.33% of unique windows (length = 100) from the generated composition exist in the training data.
sample_43-55-59-62-65.mid: 57.04% of unique windows (length = 100) from the generated composition exist in the training data.
sample_79.mid: 88.68% of unique windows (length = 100) from the generated composition exist in the training data.
sample_42.mid: 34.02% of unique windows (length = 100) from the generated composition exist in the training data.
AVERAGE PERCENTAGE OVER ALL FILES: 47.08%
********************************************************************************
TEMPERATURE: 1.0
sample_45-60-64-69.mid: 57.69% of unique windows (length = 100) from the generated composition exist in the training data.
samp

# Analysis

The results of this test are less clear. Raising the temperature does make it more likely that sequences will be more unique, but there are still outliers on either end of the spectrum.

A subjective listening test makes it clear that at a certain point, raising the temperature high enough leads to relatively incoherent sequences being generated.

Choosing the temperature helps balance the likelihood of a more realistic sounding sequence with the probablity that a sequence will be too similar to compositions from the training set.